In [13]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns

Challenge: Feedback analysis
 Estimated Time: 2-3 hours

Naive Bayes is particularly good for text classification problems. Before we made a spam filter. 

Now we'll perform a sentiment analysis, classifying whether feedback left on a website is either positive or negative.

# Building a Classifier
using the Yelp data in this file
Accuracy measure

In [83]:
# Grab and process the raw data.
#data_path = ("https://github.com/bar10009/Thinkful-Bootcamp/blob/master/yelp_labelled")
#sms_raw = pd.read_csv(data_path, delimiter= '\t', header=None)
#sms_raw.columns = ['spam', 'message']

In [84]:
df = pd.read_csv("/Users/bradleyrhyins/Desktop/Data Science/yelp_labelled.txt", sep=r'\\t',header=None)
#df.columns=['review', 'label']

/Users/bradleyrhyins/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


Let’s get some basic information about the data. The .shape method tells us the number of rows and columns in the dataframe.

In [18]:
df.shape

(1000, 1)

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 1 columns):
0    1000 non-null object
dtypes: object(1)
memory usage: 7.9+ KB


In [20]:
df.columns= ["text"]

In [22]:
label = [np.int(s.split("\t")[1]) for s in df["text"]]

In [23]:
text = [s.split(".\t")[0] for s in df["text"]]

In [24]:
DF = pd.DataFrame({"text": text, "label": label})

In [25]:
DF['text length'] = DF['text'].apply(len)
DF.head()

,text,label,text length
0,Wow... Loved this place,1,23
1,Crust is not good,0,17
2,Not tasty and the texture was just nasty,0,40
3,Stopped by during the late May bank holiday of...,1,86
4,The selection on the menu was great and so wer...,1,58


In [86]:
print (DF.text)

0                                Wow... Loved this place
1                                      Crust is not good
2               Not tasty and the texture was just nasty
3      Stopped by during the late May bank holiday of...
4      The selection on the menu was great and so wer...
                             ...                        
995    I think food should have flavor and texture an...
996                              Appetite instantly gone
997    Overall I was not impressed and would not go back
998    The whole experience was underwhelming, and I ...
999    Then, as if I hadn't wasted enough of my life ...
Name: text, Length: 1000, dtype: object


In [26]:
DF.columns

Index(['text', 'label', 'text length'], dtype='object')

In [ ]:
keywords = ['impressive', 'good', 'very good', 'better', 'love', 'loved', 'great']
#Increase the number of keywords
#Trying to predict positive
#Increase number of features

for key in keywords:
    # Note that we add spaces around the key so that we're getting the word,
    # not just pattern matching.
    DF[str(key)] = DF.text.str.contains(
        ' ' + str(key) + ' ',
        case=False
    )

In [89]:
DF.head()

,text,label,text length,impressive,good,very good,better,love,loved,great
0,Wow... Loved this place,1,23,False,False,False,False,False,True,False
1,Crust is not good,0,17,False,False,False,False,False,False,False
2,Not tasty and the texture was just nasty,0,40,False,False,False,False,False,False,False
3,Stopped by during the late May bank holiday of...,1,86,False,False,False,False,False,True,False
4,The selection on the menu was great and so wer...,1,58,False,False,False,False,False,False,True


In [90]:
DF.columns
#features

list_features = DF.columns[2:]
data = DF[list_features]
target = DF["label"]

In [91]:
print (DF.columns)

Index(['text', 'label', 'text length', 'impressive', 'good', 'very good',
       'better', 'love', 'loved', 'great'],
      dtype='object')


In [92]:
print (list_features)

Index(['text length', 'impressive', 'good', 'very good', 'better', 'love',
       'loved', 'great'],
      dtype='object')


In [93]:
# Our data is binary / boolean, so we're importing the Bernoulli classifier.
from sklearn.naive_bayes import BernoulliNB

In [94]:
# Instantiate our model and store it in a new variable.
bnb = BernoulliNB()

In [95]:
# Fit our model to the data.
bnb.fit(data, target)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [96]:
# Classify, storing the result in a new variable.
y_pred = bnb.predict(data)

In [97]:
# Display our results.
print("Number of mislabeled points out of a total {} points : {}".format(
    data.shape[0],
    (target != y_pred).sum()
))

Number of mislabeled points out of a total 1000 points : 416


## Success Rate (cross val score) measuring for accuracy

In [98]:
from sklearn.model_selection import cross_val_score
cross_val_score(bnb, data, target, cv=10)

array([0.61, 0.59, 0.59, 0.58, 0.56, 0.58, 0.57, 0.59, 0.54, 0.63])

## Confusion Matrix (shows us where we went wrong and where we went right)

In [99]:
#False negative 399 False positive 17 True negative is 483 and True positive is 101
from sklearn.metrics import confusion_matrix
confusion_matrix(target, y_pred)

array([[483,  17],
       [399, 101]])

## What we learn from the confusion matrix

Firstly, when we talk about errors in a binary classifier (where there are only two outcomes) we're generally referring to two kinds of errors. 

A __false positive__ is when we identify something as spam that is not. In this case we had 55 of these. This is sometimes also called a "Type I Error" or a "false alarm".

A __false negative__ is when we mistakenly identify something as not something we thought it was in our case (spam) when it is. We had 549 of these. This is also called a "Type II Error" or a "miss".

This brings us to a conversation of sensitivity vs specificity.

__Sensitivity__ is the percentage of positives correctly identified, in our case 198/747 or 27%. This shows how good we are at catching positives, or how sensitive our model is to identifying positives.

__Specificity__ is just the opposite, the percentage of negatives correctly identified, 4770/4825 or 99%.

In [103]:
from sklearn.model_selection import train_test_split
# Use train_test_split to create the necessary training and test groups
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.2, random_state=20)
print('With 20% Holdout: ' + str(bnb.fit(X_train, y_train).score(X_test, y_test)))
print('Testing on Sample: ' + str(bnb.fit(data, target).score(data, target)))

With 20% Holdout: 0.595
Testing on Sample: 0.584


Out of 1000 points we labeled 416 point not accurate and 584 points accurrate 
The model is 58.4% accurate

#Try to improve using more features 